In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

In [ ]:
#Spaceship Titanic has begun
import pandas as pd

df_train = pd.read_csv(r'C:\Users\omarf\Downloads\spaceship-titanic\train.csv')
df_test = pd.read_csv(r'C:\Users\omarf\Downloads\spaceship-titanic\test.csv')  
df_test['Transported'] = False

df = pd.concat([df_train, df_test], sort = False)
df.head()


In [ ]:
import pandas as pd

# Example: Loading your dataset (replace 'your_dataset.csv' with your actual file)
df = pd.read_csv(r'C:\Users\omarf\Downloads\spaceship-titanic\train.csv')

# Step 1: Check if 'CryoSleep' column exists
if 'CryoSleep' in df.columns:
    print("CryoSleep column found!")

    # Step 2: Handle missing values and normalize data
    # Convert all values to booleans or None (if they are not already)
    df['CryoSleep'] = df['CryoSleep'].astype(str).str.strip().str.lower()  # Normalize to lowercase strings
    df['CryoSleep'] = df['CryoSleep'].map({'true': True, 'false': False, 'nan': None})  # Map to boolean

    # Optional: Fill missing values with False (or keep as None, depending on your requirements)
    df['CryoSleep'].fillna(False, inplace=True)

    # Step 3: Check unique values to verify cleanup
    print("Unique values in 'CryoSleep':", df['CryoSleep'].unique())

    # Example usage: Filter rows where CryoSleep is True
    cryosleep_passengers = df[df['CryoSleep'] == True]
    print(f"Number of passengers in CryoSleep: {len(cryosleep_passengers)}")

else:
    print("CryoSleep column not found in the dataset!")


In [ ]:
df.shape[0] == df_train.shape[0] + df_train.shape[0]

In [ ]:
df.isnull().sum()

In [ ]:
df[['Deck','Num','Side']] = df['Cabin'].str.split('/', expand=True)
columns_to_drop = ['Name', 'Cryosleep']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1, inplace=True)
df['Deck'] = df['Deck'].fillna('U')
df['Side'] = df['Side'].fillna('U')
df['Num'] = df['Num'].fillna(-1)

df.head()

In [7]:
df['Deck'] = df['Deck'].map({'G':0, 'F':1, 'E':2, 'D':3, 'C':4, 'B':5, 'A':6, 'U':7, 'T':8})
df['Side'] = df['Side'].map({'U':-1, 'P':1, 'S':2})

In [ ]:
impute_list = ['Age','Num','Side','CryoSleep','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Deck','VIP']
rest = list(set(df.columns) - set(impute_list))
df_rest = df[rest]
imp = KNNImputer()
df_imputed = imp.fit_transform(df[impute_list])
df_imputed = pd.DataFrame(df_imputed, columns=impute_list)
df = pd.concat([df_rest.reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)
df.head()

In [ ]:
df['HomePlanet'] = df['HomePlanet'].fillna('U')
df['Destination'] = df['Destination'].fillna('U')
df['Cabin'] = df['Cabin'].fillna('U')

category_colls = ['HomePlanet','Destination']

for col in category_colls:
   df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)

df.drop(columns=category_colls, inplace=True)
df.head()

In [10]:
#Feature Engineering
bill_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
df['amt_spent'] = df[bill_cols].sum(axis=1)
df['std_amt_spent'] = df[bill_cols].std(axis=1)
df['mean_amt_spent'] = df[bill_cols].mean(axis=1)

df['3_High_colls'] = df['CryoSleep'] + df['HomePlanet_Europa'] + df['Destination_55 Cancri e']
df['3_low_colls'] = df['amt_spent'] + df['mean_amt_spent'] + df['HomePlanet_Earth']

In [ ]:
# Ensure 'Transported' column is included in numeric_df
numeric_df = df.select_dtypes(include=[np.number])
numeric_df['Transported'] = df['Transported'].astype(float)
numeric_df.corr()['Transported'].sort_values(ascending=False)

In [ ]:
df.isnull().sum()

In [ ]:
df_train, df_test = df[:df_train.shape[0]], df[df_train.shape[0]:]
df_test = df_test.drop('Transported', axis=1)
df_train.shape, df_test.shape

In [14]:
from xgboost import XGBRFClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [15]:
X = df_train.drop(columns = 'Transported')
y = df_train['Transported']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

model_1 = LogisticRegression()
model_2 = DecisionTreeClassifier()
model_3 = RandomForestClassifier()
model_4 = XGBRFClassifier()
model_5 = LGBMClassifier()

In [ ]:
# Ensure all features are numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(-1)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(-1)

model_1.fit(X_train, y_train)
pred = model_1.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
model_2.fit(X_train, y_train)
pred = model_2.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
model_3.fit(X_train, y_train)
pred = model_3.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
model_4.fit(X_train, y_train)
pred = model_4.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
model_5.fit(X_train, y_train)
pred = model_5.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
# Create the predictions DataFrame

df_dummy = pd.read_csv(r'C:\Users\omarf\Downloads\spaceship-titanic\test.csv')

final = pd.DataFrame()
final['PassengerId'] = df_dummy['PassengerId']  # Ensure all 4,277 PassengerIds are included
final['Transported'] = False  # Default value for missing predictions

# Ensure 'PassengerId' is present in X_test
X_test['PassengerId'] = df.loc[X_test.index, 'PassengerId']

# Match predictions to the Passenger IDs in the full dataset
predictions_df = pd.DataFrame({'PassengerId': X_test['PassengerId'], 'Transported': pred})
final = final.merge(predictions_df, on='PassengerId', how='left')

# Fill missing predictions with default value (e.g., False)
final['Transported'] = final['Transported_y'].fillna(False).astype(bool)

# Save the final output with 4,277 rows
final.to_csv(r'C:\Users\omarf\Downloads\Submission_csv.csv', index=False)


In [ ]:
# Merge predictions back into the original test dataset
final = df_dummy[['PassengerId']].merge(predictions_df, on='PassengerId', how='left')

# After the merge, ensure only one 'Transported' column exists
final = final[['PassengerId', 'Transported']]  # Keep only these two columns

# Fill missing predictions with a default value (e.g., False)
final['Transported'] = final['Transported'].fillna(False).astype(bool)

# Save the final output
final.to_csv(r'C:\Users\omarf\Downloads\Submission_csv.csv', index=False)


In [ ]:
final.shape